In [ ]:
#  Célula 1: Instalação das Bibliotecas Necessárias
# =================================================================
# Esta célula instala as bibliotecas que não são padrão no Python/Jupyter
# e que são necessárias para a execução completa deste notebook.
# O '!' permite executar comandos do terminal diretamente na célula.
# O '-q' é para uma instalação "quieta", com menos texto de output.

!pip install -q --upgrade statsmodels seaborn openpyxl

print("Bibliotecas necessárias instaladas/verificadas com sucesso.")

In [ ]:
# Célula 2: Importação das Bibliotecas e Configurações Iniciais
# =================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, f_oneway
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.contingency_tables import Table

# Define um tema visual padrão para todos os gráficos gerados pela seaborn
sns.set_theme(style="whitegrid")

# Aumenta a resolução padrão (DPI) das figuras para melhor qualidade visual
plt.rcParams['figure.dpi'] = 100

print("Bibliotecas importadas e configurações visuais aplicadas com sucesso.")


In [ ]:
# Célula 3: Carregamento e Preparação dos Dados - VERSÃO ATUALIZADA
# =================================================================

try:
    # Tenta carregar o arquivo Excel na mesma pasta do script.
    file_path = 'banco de dados - livro - cap 2.xlsx'
    df = pd.read_excel(file_path)
    print(f"Arquivo '{file_path}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{file_path}' não foi encontrado. Verifique o caminho.")
    df = None

if df is not None:
    grupos_analise = ['Branca', 'Parda', 'Preta']
    df_analysis = df[df['Cor_Raça_TSE'].isin(grupos_analise)].copy()
    
    print(f"\nTotal de deputados considerados na análise (Brancos, Pardos, Pretos): {len(df_analysis)}")
    print("Primeiras 5 linhas do DataFrame preparado para análise:")
    display(df_analysis.head())

In [ ]:
# Célula 4: Função para Cálculo de Testes de Associação
# =================================================================
# Esta função calcula o Qui-quadrado e o V de Cramer para uma tabela de contingência.
# Ela será reutilizada em todas as análises de perfil.

def calculate_association_stats(crosstab):
    """
    Calcula e retorna uma string formatada com os resultados do teste Qui-quadrado e V de Cramer.
    """
    try:
        # Remove a linha 'Total' se ela existir, para não interferir no cálculo
        if 'Total' in crosstab.index:
            crosstab = crosstab.drop('Total')
        if 'Total' in crosstab.columns:
            crosstab = crosstab.drop('Total', axis=1)

        chi2, p, dof, expected = chi2_contingency(crosstab)
        
        # Calcula o V de Cramer
        n = crosstab.sum().sum()
        phi2 = chi2 / n
        r, k = crosstab.shape
        v_cramer = np.sqrt(phi2 / min(k - 1, r - 1))
        
        # Formata os resultados
        resultado = (f"  Qui-Quadrado (χ²): {chi2:.2f}\n"
                     f"  p-valor: {p:.4f}\n"
                     f"  V de Cramer: {v_cramer:.3f}")
        
        # Interpretação
        if p < 0.05:
            resultado += "\n  -> Associação estatisticamente significativa (p < 0.05)."
        else:
            resultado += "\n  -> Associação não é estatisticamente significativa (p >= 0.05)."
            
        return resultado
        
    except ValueError:
        return "Não foi possível calcular os testes (verifique se há dados suficientes)."

print("Função 'calculate_association_stats' criada com sucesso.")

In [ ]:
# Célula 5: Cálculo do Coeficiente de Evolução da Representação (CER)
# =================================================================
# O cálculo foi feito para analisar Brancos, Pardos e Pretos separadamente,
# e um grupo agregado de 'Não Brancos' foi adicionado.

# Contagem de deputados por ano e grupo racial
counts_per_year = df_analysis.groupby(['Ano_Eleição', 'Cor_Raça_TSE']).size().unstack(fill_value=0)

# Garante que todas as colunas de análise existam para evitar KeyErrors
for col in ['Branca', 'Parda', 'Preta']:
    if col not in counts_per_year.columns:
        counts_per_year[col] = 0

# Cria a coluna agregada de 'Nao_Branca'
counts_per_year['Nao_Branca'] = counts_per_year['Parda'] + counts_per_year['Preta']

# Define a base de 2014 para o cálculo
try:
    base_2014 = counts_per_year.loc[2014]
    print("Valores de base para 2014 que o script está usando:")
    print(base_2014)

    # Função segura para calcular o CER, que lida com base zero
    def safe_cer_calc(series, base_value):
        if base_value == 0:
            # Retorna uma série de infinitos se a base for 0
            # O ano base (2014) será ajustado para 1.0 posteriormente
            return pd.Series(np.inf, index=series.index)
        return series / base_value

    # Calcula o CER para cada grupo usando a função segura
    cer_b = safe_cer_calc(counts_per_year['Branca'], base_2014['Branca'])
    cer_p = safe_cer_calc(counts_per_year['Parda'], base_2014['Parda'])
    cer_pr = safe_cer_calc(counts_per_year['Preta'], base_2014['Preta'])
    cer_nb = safe_cer_calc(counts_per_year['Nao_Branca'], base_2014['Nao_Branca'])

    # Monta o DataFrame final com os novos resultados do CER
    cer_results = pd.DataFrame({
        'CER (Brancos)': cer_b,
        'CER (Pardos)': cer_p,
        'CER (Pretos)': cer_pr,
        'CER (Não Brancos)': cer_nb
    })
    
    # Ajusta o ano base para ser 1.0 em todos os casos
    cer_results.loc[2014] = 1.0

    print("Contagem de deputados por grupo e ano (incluindo 'Não Branca'):")
    display(counts_per_year)

    print("\nResultados do Coeficiente de Evolução da Representação (CER):")
    display(cer_results.round(2))

except KeyError:
    print("ERRO: O ano base de 2014 não foi encontrado nos dados. A análise do CER não pode prosseguir.")
    # Cria um DataFrame vazio para evitar que as células seguintes quebrem
    cer_results = pd.DataFrame()

In [ ]:
# Célula 5: Gráfico 1 - Evolução do CER (Gráfico de Linhas)
# =================================================================

plt.figure(figsize=(10, 6))
sns.lineplot(data=cer_results, markers=True, dashes=False, palette="viridis")
plt.title('Gráfico 1: Coeficiente de Evolução da Representação (CER)', fontsize=16)
plt.xlabel('Ano da Eleição', fontsize=12)
plt.ylabel('Coeficiente (Base 2014 = 1.0)', fontsize=12)
plt.xticks(cer_results.index)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(title='Grupo Racial')
plt.tight_layout()

plt.savefig('grafico_1_cer_evolucao_atualizado.png')
plt.show()

In [ ]:
# Célula 6: Gráfico 2 - Composição Absoluta da Câmara (Gráfico de Barras Empilhadas)
# =================================================================

counts_to_plot = counts_per_year[['Branca', 'Parda', 'Preta']]
ax = counts_to_plot.plot(kind='bar', stacked=True, figsize=(10, 7), colormap='cividis')

for c in ax.containers:
    ax.bar_label(c, label_type='center', fmt='%d', color='white', weight='bold')

plt.title('Gráfico 2: Composição Racial Absoluta da Câmara por Legislatura', fontsize=16)
plt.xlabel('Ano da Eleição', fontsize=12)
plt.ylabel('Número de Deputados', fontsize=12)
plt.xticks(rotation=0)
plt.legend(title='Grupo Racial')
plt.tight_layout()

plt.savefig('grafico_2_composicao_absoluta_atualizado.png')
plt.show()

In [ ]:
# Célula 7: Análise Descritiva Comparativa com Testes Estatísticos
# =================================================================
# Esta célula realiza a análise de perfil, agora incluindo os testes de associação para cada ano.

print("\n### Análise Descritiva Comparativa por Grupo Racial com Testes de Associação ###")

# Variáveis a serem analisadas
variables_to_analyze = ['Região', 'Escolaridade', 'Gênero']

# Simplifica a escolaridade para o teste
df_analysis['Escolaridade_Simples'] = df_analysis['Escolaridade'].replace({
    'Ensino Médio Completo': 'Até Ensino Médio', 'Ensino Fundamental Completo': 'Até Ensino Médio',
    'Ensino Fundamental Incompleto': 'Até Ensino Médio', 'Ensino Médio Incompleto': 'Até Ensino Médio',
    'Lê e Escreve': 'Até Ensino Médio', 'Superior Incompleto': 'Superior Incompleto',
    'Superior Completo': 'Superior Completo', 'Mestrado': 'Pós-graduação',
    'Doutorado': 'Pós-graduação', 'Especialização': 'Pós-graduação'
}).fillna('Não informado')


for year in [2014, 2018, 2022]:
    print(f"\n" + "="*50)
    print(f" ANO DE ANÁLISE: {year}")
    print("="*50)
    
    df_year = df_analysis[df_analysis['Ano_Eleição'] == year]
    
    for var in variables_to_analyze:
        column_to_use = 'Escolaridade_Simples' if var == 'Escolaridade' else var
        
        print(f"\n----- Análise da variável: {var} -----")
        
        # Cria a tabela de contingência (crosstab)
        crosstab = pd.crosstab(df_year['Cor_Raça_TSE'], df_year[column_to_use])
        
        print("\nContagem Absoluta:")
        display(crosstab)
        
        print("\nComposição Percentual (por raça):")
        crosstab_pct = crosstab.div(crosstab.sum(axis=1), axis=0).mul(100).round(1)
        display(crosstab_pct)
        
        # Calcula e exibe os testes de associação
        print("\nTestes de Associação:")
        stats_results = calculate_association_stats(crosstab)
        print(stats_results)

In [ ]:
# Célula 9: Análise da Composição Racial e Média por Ano
# =================================================================
# Esta célula analisa a variação da composição percentual de raças ao longo dos anos
# e calcula a média de deputados por grupo no período.

print("\n### Análise da Composição Racial ao Longo dos Anos ###")

# Tabela de contagem (já calculada como 'counts_per_year')
print("\nContagem Absoluta de Deputados por Raça e Ano:")
display(counts_per_year)

# Calcula o total de deputados em cada ano para a base percentual
total_por_ano = counts_per_year.sum(axis=1)
composition_percentage = counts_per_year.div(total_por_ano, axis=0).mul(100)

print("\nComposição Percentual de Deputados por Raça e Ano (%):")
display(composition_percentage.round(1))

# Teste estatístico para ver se a distribuição racial muda significativamente entre os anos
print("\nTestes de Associação (Ano vs. Raça):")
# O crosstab aqui é a própria tabela `counts_per_year`
print(calculate_association_stats(counts_per_year))

print("\n" + "="*50)
print("\n### Média de Deputados por Grupo Racial (2014-2022) ###")

media_deputados = counts_per_year.mean()
print("\nNúmero médio de deputados eleitos por ano para cada grupo:")
display(media_deputados.round(1).to_frame(name='Média por Ano'))

In [ ]:
# Célula 10: Validação Estatística da Tendência com Teste Cochran-Armitage - VERSÃO CORRIGIDA
# =================================================================
# Esta célula implementa o teste estatístico mais adequado para validar a tendência
# observada no CER: o Teste Cochran-Armitage para Tendência.
# Este teste verifica se a proporção de deputados de um grupo mostra uma
# tendência linear de aumento ou queda ao longo dos anos.

print("\n### Validação Estatística da Tendência de Representação ###")

# --- Teste de Tendência para a proporção de Deputados Não Brancos ---
print("\n--- Teste de Tendência para a proporção de Deputados Não Brancos ---")

# Prepara a tabela de contingência para o teste
# As linhas são os anos (categorias ordenadas)
# As colunas são os grupos a serem comparados (Não Brancos vs. Brancos)
table_for_trend_test = counts_per_year[['Nao_Branca', 'Branca']].copy()

print("\nTabela de Contingência para o Teste de Tendência:")
display(table_for_trend_test)

# Realiza o teste Cochran-Armitage usando a classe Table
# Esta é uma abordagem mais robusta para diferentes versões da statsmodels
table = Table(table_for_trend_test)
result = table.test_ordinal_association()

# Exibe os resultados
print(f"\nEstatística do teste: {result.statistic:.4f}")
print(f"P-valor: {result.pvalue:.4f}")

if result.pvalue < 0.05:
    print("\nConclusão: A tendência de aumento na proporção de deputados não brancos é estatisticamente significativa (p < 0.05).")
else:
    print("\nConclusão: A tendência de aumento na proporção de deputados não brancos não é estatisticamente significativa (p >= 0.05).")


In [ ]:
# Célula 11: Gráfico 3 - Composição Absoluta de Gênero por Grupo Racial
# =================================================================
# Este gráfico visualiza a contagem de homens e mulheres dentro de cada grupo racial.

# Calcula a contagem absoluta de gênero
gender_counts = df_analysis.groupby(['Ano_Eleição', 'Cor_Raça_TSE'])['Gênero'].value_counts().unstack(fill_value=0)

# Cria os subplots para cada grupo
groups = gender_counts.index.get_level_values('Cor_Raça_TSE').unique()
fig, axes = plt.subplots(1, len(groups), figsize=(15, 6), sharey=True)
fig.suptitle('Gráfico 3: Composição Absoluta de Gênero por Grupo Racial', fontsize=16)

for i, group in enumerate(groups):
    ax = axes[i]
    data_to_plot = gender_counts.loc[(slice(None), group), :]
    data_to_plot.index = data_to_plot.index.get_level_values('Ano_Eleição')
    data_to_plot.plot(kind='bar', stacked=True, ax=ax, colormap='coolwarm', legend=(i == len(groups) - 1))
    ax.set_title(f'Grupo: {group}')
    ax.set_xlabel('Ano')
    ax.tick_params(axis='x', rotation=0)
    for c in ax.containers:
        labels = [f'{v.get_height():.0f}' if v.get_height() > 0 else '' for v in c]
        ax.bar_label(c, labels=labels, label_type='center', color='white', weight='bold')

axes[0].set_ylabel('Número de Deputados')
if fig.legends:
    fig.legends[0].set_title('Gênero')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('grafico_3_genero_raca_absoluto.png')
plt.show()

In [ ]:
# Célula 10: Gráfico 4 - Composição Regional por Ano
# =================================================================
# Mostra a composição racial de cada região, com um subplot para cada ano.

regional_counts = df_analysis.groupby(['Ano_Eleição', 'Região', 'Cor_Raça_TSE']).size().unstack(fill_value=0)
regional_counts = regional_counts[['Branca', 'Parda', 'Preta']]

years = regional_counts.index.get_level_values('Ano_Eleição').unique()
fig, axes = plt.subplots(1, 3, figsize=(20, 7), sharey=True)
fig.suptitle('Gráfico 4: Composição Racial por Região e Ano', fontsize=16)

for i, year in enumerate(years):
    ax = axes[i]
    data_to_plot = regional_counts.loc[year]
    data_to_plot.plot(kind='bar', stacked=True, ax=ax, colormap='cividis', legend=(i == 2))
    ax.set_title(f'Ano: {year}')
    ax.set_xlabel('Região')
    ax.tick_params(axis='x', rotation=45)
    for c in ax.containers:
        labels = [v.get_height() if v.get_height() > 0 else '' for v in c]
        ax.bar_label(c, labels=labels, label_type='center', fmt='%d', color='white', weight='bold')

axes[0].set_ylabel('Número de Deputados')
if fig.legends:
    fig.legends[0].set_title('Grupo Racial')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('grafico_4_composicao_regional_por_ano.png')
plt.show()

In [ ]:
# Célula 11: Gráfico 5 - Distribuição de Idade em 2022 (Box Plot)
# =================================================================
# Compara a distribuição de idade dos deputados eleitos em 2022.

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_analysis[df_analysis['Ano_Eleição'] == 2022], x='Cor_Raça_TSE', y='Idade_Posse',
            order=['Branca', 'Parda', 'Preta'])
plt.title('Gráfico 5: Distribuição de Idade na Posse por Grupo Racial (2022)', fontsize=16)
plt.xlabel('Grupo Racial', fontsize=12)
plt.ylabel('Idade na Posse', fontsize=12)
plt.tight_layout()
plt.savefig('grafico_5_distribuicao_idade_2022.png')
plt.show()

In [ ]:
# Célula 14: Exportação das Tabelas para Reprodução dos Gráficos
# =================================================================
# Salva as tabelas exatas usadas em cada gráfico em arquivos Excel.

print("\n### Preparando e salvando tabelas para reprodução dos gráficos... ###")

# Tabela para o Gráfico 1 (Evolução do CER)
cer_results.to_excel('dados_grafico_1_cer_evolucao.xlsx')
print("- Salvo: dados_grafico_1_cer_evolucao.xlsx")

# Tabela para o Gráfico 2 (Composição Absoluta)
counts_per_year[['Branca', 'Parda', 'Preta']].to_excel('dados_grafico_2_composicao_absoluta.xlsx')
print("- Salvo: dados_grafico_2_composicao_absoluta.xlsx")

# Tabela para o Gráfico 3 (Gênero e Raça - Absoluto)
gender_counts = df_analysis.groupby(['Ano_Eleição', 'Cor_Raça_TSE'])['Gênero'].value_counts().unstack(fill_value=0)
gender_counts.to_excel('dados_grafico_3_genero_raca_absoluto.xlsx')
print("- Salvo: dados_grafico_3_genero_raca_absoluto.xlsx")

# Tabela para o Gráfico 4 (Composição Regional por Ano)
regional_counts.to_excel('dados_grafico_4_composicao_regional.xlsx')
print("- Salvo: dados_grafico_4_composicao_regional.xlsx")

# Tabela para o Gráfico 5 (Distribuição de Idade 2022)
df_analysis[df_analysis['Ano_Eleição'] == 2022][['Nome_Deputado', 'Cor_Raça_TSE', 'Idade_Posse']].to_excel('dados_grafico_5_idade_2022.xlsx', index=False)
print("- Salvo: dados_grafico_5_idade_2022.xlsx")

print("\nTodas as tabelas para os gráficos foram salvas com sucesso!")

In [ ]:
# Célula 15: Análise Descritiva Comparativa
# =================================================================
# Expande a análise descritiva para mais variáveis, com testes estatísticos.

print("\n### Análise Descritiva Comparativa Aprofundada (com Testes de Associação) ###")

# Variáveis para a análise aprofundada
variables_deep_dive = ['Região', 'Escolaridade_Simples', 'Gênero', 'Sigla_Partido', 'Ocupação_Profissional']

for year in [2014, 2018, 2022]:
    print(f"\n" + "="*60)
    print(f" ANO DE ANÁLISE: {year}")
    print("="*60)
    
    df_year = df_analysis[df_analysis['Ano_Eleição'] == year]
    
    # Análise da Idade (variável numérica)
    print("\n----- Análise da variável: Idade -----")
    age_summary = df_year.groupby('Cor_Raça_TSE')['Idade_Posse'].agg(['mean', 'median', 'std']).round(1)
    print("Resumo estatístico da Idade na Posse:")
    display(age_summary)
    
    # ***NOVO*** Teste ANOVA para comparar as médias de idade entre os grupos raciais
    age_groups = [df_year['Idade_Posse'][df_year['Cor_Raça_TSE'] == group] for group in df_year['Cor_Raça_TSE'].unique()]
    # Garante que há mais de um grupo para comparar
    if len(age_groups) > 1:
        f_stat, p_value = f_oneway(*age_groups)
        print(f"Teste ANOVA (diferença de médias de idade): p-valor = {p_value:.4f}")
        if p_value < 0.05:
            print("  -> A diferença entre as médias de idade é estatisticamente significativa.")
        else:
            print("  -> A diferença entre as médias de idade não é estatisticamente significativa.")

    
    for var in variables_deep_dive:
        print(f"\n----- Análise da variável: {var} -----")
        
        # Para variáveis com muitas categorias, analisamos as Top 10 para clareza
        if var in ['Sigla_Partido', 'Ocupação_Profissional']:
            top_categories = df_year[var].value_counts().nlargest(10).index
            df_var_filtered = df_year[df_year[var].isin(top_categories)]
            crosstab = pd.crosstab(df_var_filtered['Cor_Raça_TSE'], df_var_filtered[var])
        else:
            crosstab = pd.crosstab(df_year['Cor_Raça_TSE'], df_year[var])
            
        print("\nContagem Absoluta:")
        display(crosstab)
        
        print("\nTestes de Associação:")
        stats_results = calculate_association_stats(crosstab)
        print(stats_results)

In [ ]:
# Célula 16: Análise Interseccional Focada em Gênero
# =================================================================
# Esta célula aprofunda a análise de gênero, cruzando com raça e outras variáveis.

print("\n### Análise Interseccional Detalhada (Gênero e Raça) ###")

# --- 1. Composição Racial DENTRO de cada Gênero ---
print("\n--- 1. Composição Racial por Gênero (Total do Período) ---")
# Tabela de contagem para o teste
gender_race_crosstab = pd.crosstab(df_analysis['Gênero'], df_analysis['Cor_Raça_TSE'])
print("\nContagem Absoluta:")
display(gender_race_crosstab)

# Tabela de porcentagem
gender_race_composition_pct = gender_race_crosstab.div(gender_race_crosstab.sum(axis=1), axis=0).mul(100).round(1)
print("\nComposição racial percentual dentro de cada gênero:")
display(gender_race_composition_pct)

# Teste de associação para a tabela geral de gênero vs. raça
print("\nTestes de Associação (Gênero vs. Raça):")
print(calculate_association_stats(gender_race_crosstab))


# --- 2. Análise de Disparidade (Brancos vs. Não Brancos por Gênero) com Teste de Significância ---
print("\n--- 2. Análise da Disparidade de Representação por Gênero com Teste de Significância ---")

# Agrupa Brancos vs. Pardos+Pretos
df_analysis['Grupo_Binario'] = df_analysis['Cor_Raça_TSE'].apply(lambda x: 'Brancos/as' if x == 'Branca' else 'Não Brancos/as')

# Calcula contagens absolutas para o teste Z
disparity_counts = pd.crosstab(index=[df_analysis['Ano_Eleição'], df_analysis['Gênero']], columns=df_analysis['Grupo_Binario'])

# Calcula a tabela de porcentagens
disparity_table = disparity_counts.div(disparity_counts.sum(axis=1), axis=0).mul(100)
disparity_table['Diferença (pp)'] = disparity_table['Brancos/as'] - disparity_table['Não Brancos/as']

# Realiza o Teste Z para a diferença de proporções e adiciona o p-valor
p_values = []
for index, row in disparity_counts.iterrows():
    count = [row['Brancos/as'], row['Não Brancos/as']]
    nobs = [row.sum(), row.sum()] # O total é o mesmo para ambos dentro da mesma categoria de gênero
    stat, pval = proportions_ztest(count=count, nobs=nobs, alternative='two-sided')
    p_values.append(pval)

disparity_table['p-valor (Teste Z)'] = p_values
disparity_table['Significativo'] = disparity_table['p-valor (Teste Z)'] < 0.05

print("\nComposição percentual, diferença (pp) e significância estatística (Brancos vs. Não Brancos):")
display(disparity_table.round(3))


# --- 3. Análise do Perfil das Mulheres por Raça  ---
print("\n\n--- 3. Análise Comparativa do Perfil das Deputadas Eleitas por Raça ---")
df_mulheres = df_analysis[df_analysis['Gênero'] == 'Feminino'].copy()

# 3.1 Evolução da quantidade de deputadas por raça
print("\n--- 3.1 Evolução da Composição da Bancada Feminina por Raça ---")
women_counts_over_time = pd.crosstab(df_mulheres['Ano_Eleição'], df_mulheres['Cor_Raça_TSE'])
print("\nContagem de deputadas por raça ao longo dos anos:")
display(women_counts_over_time)
print("\nTeste de Associação (Ano vs. Raça para Mulheres):")
print(calculate_association_stats(women_counts_over_time))


for year in [2014, 2018, 2022]:
    print(f"\n" + "="*50)
    print(f" ANO DE ANÁLISE: {year} (Mulheres)")
    print("="*50)
    
    df_mulheres_year = df_mulheres[df_mulheres['Ano_Eleição'] == year]
    
    # 3.2 Perfil de Idade
    print("\nPerfil de Idade das Deputadas:")
    age_summary_women = df_mulheres_year.groupby('Cor_Raça_TSE')['Idade_Posse'].agg(['mean', 'median', 'std']).round(1)
    display(age_summary_women)
    # Teste ANOVA para comparar as médias de idade
    age_groups = [df_mulheres_year['Idade_Posse'][df_mulheres_year['Cor_Raça_TSE'] == group] for group in df_mulheres_year['Cor_Raça_TSE'].unique()]
    # Garante que há mais de um grupo para comparar
    if len(age_groups) > 1:
        f_stat, p_value = f_oneway(*age_groups)
        print(f"Teste ANOVA (diferença de médias de idade): p-valor = {p_value:.4f}")
        if p_value < 0.05:
            print("  -> A diferença entre as médias de idade é estatisticamente significativa.")
        else:
            print("  -> A diferença entre as médias de idade não é estatisticamente significativa.")

    # 3.3 Perfil de Escolaridade
    print("\nPerfil de Escolaridade das Deputadas:")
    crosstab_escolaridade = pd.crosstab(df_mulheres_year['Cor_Raça_TSE'], df_mulheres_year['Escolaridade_Simples'])
    display(crosstab_escolaridade)
    print("Testes de Associação (Escolaridade):")
    print(calculate_association_stats(crosstab_escolaridade))
    
    # 3.4 Perfil Regional
    print("\nPerfil Regional das Deputadas:")
    crosstab_regiao = pd.crosstab(df_mulheres_year['Cor_Raça_TSE'], df_mulheres_year['Região'])
    display(crosstab_regiao)
    print("Testes de Associação (Região):")
    print(calculate_association_stats(crosstab_regiao))
    
    # 3.5 Perfil Partidário (Top 10 partidos com mais mulheres)
    print("\nPerfil Partidário das Deputadas (Top 10 Partidos):")
    top_parties_women = df_mulheres_year['Sigla_Partido'].value_counts().nlargest(10).index
    df_mulheres_top_parties = df_mulheres_year[df_mulheres_year['Sigla_Partido'].isin(top_parties_women)]
    crosstab_partido = pd.crosstab(df_mulheres_top_parties['Cor_Raça_TSE'], df_mulheres_top_parties['Sigla_Partido'])
    display(crosstab_partido)
    print("Testes de Associação (Partido):")
    print(calculate_association_stats(crosstab_partido))